![](https://www.saa-authors.eu/picture/739/ftw_768/saa-mtcwmza4nzq5mq.jpg)

## How will teaching happen with the Corona lock-down?

We will do it virtually just like today. Before the lecture starts we will share a link to Zoom (similar to: `https://itucph.zoom.us/<letter>/<number>`) via https://talk.itu.dk/channel/itu-devops-all.

Likely teaching style will change a bit, i.e., your group work exercises... Let's see how it works today and then we will try to adjust.

For your group project work find similar means of collaborating remotely on the projects. Likely, you have established such ways already, for example via talk.itu.dk/Slack/Skype/Discord/... and your CI/CD chains together with VCS and issue tracker, etc.

# Recap


## How is it going with your projects?

## The state of your projects?

http://142.93.104.18/status.html

  * http://64.225.103.230/commit_activity_weekly.svg
  * http://64.225.103.230/release_activity_weekly.svg

## Monitoring?


Only half of you send a pull-request to https://github.com/itu-devops/2020-spring/blob/master/misc_urls.py with a monitoring URL?

![](images/group_h_monitoring.png)

![](images/group_i_monitoring.png)

![](images/group_g_monitoring.png)

### Please display some information on your monitoring dashboards!

![](images/group_a_monitoring.png)

### Please remember that the monitoring dashboard has to be accessible for us...

Either it is public or you have to use the credentials from the last session that were also distributed via https://talk.itu.dk/channel/itu-devops-all.

![](images/group_k_monitoring.png)

![](images/group_l_monitoring.png)

## Maintenance?

Did you observe something after setting up your monitoring that you fixed?

### Your Turn! -  `Task 1`
<img src="https://media.giphy.com/media/13GIgrGdslD9oQ/giphy.gif" width=50%/>

  - Map the all subsystems (components) of your _ITU-MiniTwit_ in a UML component diagram.
  - That is, map all those parts that are either deployable separately or that contain bigger logical groups of functionality.
  
  
  - If rusty on component diagrams, you may want to check one of the following websites:
   * https://developer.ibm.com/articles/the-component-diagram/
   * http://www.agilemodeling.com/artifacts/componentDiagram.htm.
   * https://en.wikipedia.org/wiki/Component_diagram

### How it may look from the simulator

![](images/inclass_component.png)

## Kinds of Interfaces

  * Within a single programming language
  * Across programming languages
  * Over a network
    - Web API
    - REST API
    - SOA webservice
    - etc.

### Your Turn! -  `Task 2`
<img src="https://media.giphy.com/media/13GIgrGdslD9oQ/giphy.gif" width=50%/>

  - For each of the identified interfaces in your systems, map their kind, i.e., is it:
    * Within a single programming language
    * Across programming languages
    * Over a network
      - Web API
      - REST API
      - SOA webservice
      - etc.
  - Choose one of the identified interfaces (perhaps the most central) and try to specify the interface, i.e.:
    - What can be called?
    - In which way?
    - How doe data need to be provided?
    - How is data returned

### Was that an easy task?

We experienced this problem earlier:
    
![](images/inclass_component2.png)

#### What did we have to do to make that setup work?

We had to:

  * Provide an example specification encoded in a program [`minitwit_sim_api.py`](https://github.com/itu-devops/2020-spring/blob/master/sessions/session_03/API%20Spec/minitwit_sim_api.py).
  * Provide a unit test [`minitwit_sim_api_test.py`](https://github.com/itu-devops/2020-spring/blob/master/sessions/session_03/API%20Spec/minitwit_sim_api_test.py) to demonstrate how the API is called, with which parameters and with which data.
  * Provide an example client [`minitwit_simulator.py`](https://github.com/itu-devops/2020-spring/blob/master/sessions/session_03/API%20Spec/minitwit_simulator.py) with example data  ([`minitwit_scenario.csv`](https://github.com/itu-devops/2020-spring/blob/master/sessions/session_03/API%20Spec/minitwit_scenario.csv)) to allow for an integration test

##### Excerpt from [`minitwit_sim_api.py`](https://github.com/itu-devops/2020-spring/blob/master/sessions/session_03/API%20Spec/minitwit_sim_api.py)
```python
@app.route("/msgs", methods=["GET"])
def messages():
    no_msgs = request.args.get("no", type=int, default=100)
    if request.method == "GET":
        messages = query_db(...)

        filtered_msgs = []
        for msg in messages:
            filtered_msg = {}
            filtered_msg["content"] = msg["text"]
            filtered_msg["pub_date"] = msg["pub_date"]
            filtered_msg["user"] = msg["username"]
            filtered_msgs.append(filtered_msg)

        return jsonify(filtered_msgs)


@app.route("/fllws/<username>", methods=["GET", "POST"])
def follow(username):

    user_id = get_user_id(username)
    if not user_id:
        abort(404)

    if request.method == "POST" and "follow" in request.json.keys():
        follows_user_id = get_user_id(request.json["follow"])
        if not follows_user_id:
            abort(404)
        return "", 204

    elif request.method == "POST" and "unfollow" in request.json.keys():
        unfollows_user_id = get_user_id(request.json["unfollow"])
        if not unfollows_user_id:
            abort(404)
        return "", 204

    elif request.method == "GET":
        no_followers = request.args.get("no", type=int, default=100)
        ...
        followers_response = {"follows": follower_names}
        return jsonify(followers_response)
```

##### Excerpt from [`minitwit_sim_api_test.py`](https://github.com/itu-devops/2020-spring/blob/master/sessions/session_03/API%20Spec/minitwit_sim_api_test.py)

```python
def test_get_latest_user_msgs():
    username = 'a'

    query = {'no': 20, 'latest': 3}
    url = f'{BASE_URL}/msgs/{username}'
    response = requests.get(url, headers=HEADERS, params=query)
    assert response.status_code == 200

    got_it_earlier = False
    for msg in response.json():
        if msg['content'] == 'Blub!' and msg['user'] == username:
            got_it_earlier = True

    assert got_it_earlier
    
    
def test_follow_user():
    username = 'a'
    url = f'{BASE_URL}/fllws/{username}'
    data = {'follow': 'b'}
    params = {'latest': 7}
    response = requests.post(url, data=json.dumps(data),
                             headers=HEADERS, params=params)
    assert response.ok

    data = {'follow': 'c'}
    params = {'latest': 8}
    response = requests.post(url, data=json.dumps(data),
                             headers=HEADERS, params=params)
    assert response.ok

    json_data = response.json()
    assert "b" in json_data["follows"]
    assert "c" in json_data["follows"]
```

### Your Turn! -  `Task 3`
<img src="https://media.giphy.com/media/13GIgrGdslD9oQ/giphy.gif" width=50%/>

Reflect with your group mates why the above setup is not suitable in a DevOps environment.

DevOps focuses on shortening deployment lead time to a period of minutes. 

![](https://2wded2jn6ekmzt6b3ruvnh15-wpengine.netdna-ssl.com/wp-content/uploads/2017/10/Leadtime-high-res-768x289.png)

  * Eliminate Hardships and Waste in the Value Stream
  * Enable Optimizing for Downstream Work Centers

## The problem:

We had to do all this for only one single API.

Imagine our system grows and we would have to do that for all interfaces between subsystems.

That is unpractical since it:

  * is error prone
  * is slow, for every change in the API we have to:
    - run a lot of tests and
    - communicate and coordinate a lot with potentially remote groups
  
Consequently, exchanging components becomes a hard task.

## Kinds of APIs

  * Within a single programming language
  * Across programming languages
  * Over a network
    - Web API
    - REST API
    - SOA webservice
    - etc.

## Example of an API Within a Single Programming Language API

In the very beginning we had this `flag_tool.c` program. A C program, which could flag certain tweets directly in the database:


```c
#include <sqlite3.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

char *docStr = "ITU-Minitwit Tweet Flagging Tool\n\n"
               "Usage:\n"
               "  flag_tool <tweet_id>...\n"
               "  flag_tool -i\n"
               "  flag_tool -h\n"
               "Options:\n"
               "-h            Show this screen.\n"
               "-i            Dump all tweets and authors to STDOUT.\n";

static int callback(void *data, int argc, char **argv, char **azColName) {
  printf("%s,%s,%s,%s\n", argv[0] ? argv[0] : "NULL",
         argv[1] ? argv[1] : "NULL", argv[2] ? argv[2] : "NULL",
         argv[4] ? argv[4] : "NULL");
  return 0;
}

int main(int argc, char *argv[]) {
  sqlite3 *db;
  char *zErrMsg = 0;
  int rc;
  char query[256];
  const char *data = "Callback function called";

  rc = sqlite3_open("/tmp/minitwit.db", &db);
  if (rc) {
    fprintf(stderr, "Can't open database: %s\n", sqlite3_errmsg(db));
    return (0);
  }
  if (argc == 2 && strcmp(argv[1], "-h") == 0) {
    fprintf(stdout, "%s\n", docStr);
  }
  if (argc == 2 && strcmp(argv[1], "-i") == 0) {
    strcpy(query, "SELECT * FROM message");
    /* Execute SQL statement */
    rc = sqlite3_exec(db, query, callback, (void *)data, &zErrMsg);
    if (rc != SQLITE_OK) {
      fprintf(stderr, "SQL error: %s\n", zErrMsg);
      sqlite3_free(zErrMsg);
    }
  }
  if (argc >= 2 && strcmp(argv[1], "-i") != 0 && strcmp(argv[1], "-h") != 0) {
    int i;
    for (i = 1; i < argc; i++) {
      strcpy(query, "UPDATE message SET flagged=1 WHERE message_id=");
      strcat(query, argv[i]);
      rc = sqlite3_exec(db, query, callback, (void *)data, &zErrMsg);
      if (rc != SQLITE_OK) {
        fprintf(stderr, "SQL error: %s\n", zErrMsg);
        sqlite3_free(zErrMsg);
      } else {
        printf("Flagged entry: %s\n", argv[i]);
      }
    }
  }

  sqlite3_close(db);
  return 0;
}
```


This small program relied on the SQLite3 API. It calls the functions `sqlite3_open`, `sqlite3_errmsg`, `sqlite3_exec`, `sqlite3_free`, and `sqlite3_close` and receives corresponding return values from the SQLite3 library.

The API is [introduced](https://www.sqlite.org/cintro.html) and [described in detail](https://sqlite.org/capi3ref.html).



For example, possible calls of `sqlite3_open` are defined here https://sqlite.org/capi3ref.html#sqlite3_open as:

```c
int sqlite3_open(
  const char *filename,   /* Database filename (UTF-8) */
  sqlite3 **ppDb          /* OUT: SQLite db handle */
);
int sqlite3_open16(
  const void *filename,   /* Database filename (UTF-16) */
  sqlite3 **ppDb          /* OUT: SQLite db handle */
);
int sqlite3_open_v2(
  const char *filename,   /* Database filename (UTF-8) */
  sqlite3 **ppDb,         /* OUT: SQLite db handle */
  int flags,              /* Flags */
  const char *zVfs        /* Name of VFS module to use */
);
```

Actually, the API is defined and described in a file sqlite.h:

https://github.com/sqlite/sqlite/blob/24e399038bfb59823ea6bf1939c2282eeaf4401a/src/sqlite.h.in#L3514



### Do you have a similar file for a central interface somewhere in your code base?

If not how can you quickly tell another group how to use your component?

### How to design good APIs?
Joshua Bloch [_"How to Design a Good API and Why it Matters "_](https://dl.acm.org/doi/pdf/10.1145/1176617.1176622?casa_token=RpflkPXOrDEAAAAA:E_M0ajfP1oTa8cOVAbaLMWRCeoguv5TV6E2ka71em0i4h_BUDLjyWFYWz6Q0RZwjJsifAaG4cdw) and the [corresponding presentation slides](http://static.googleusercontent.com/media/research.google.com/en//pubs/archive/32713.pdf)

## Example of a Web API

That example uses an OpenAPI Specification (OAS), a declarative specification of all operations, data, and responses of an interface.

![](images/oas.png)

  > Arnaud Lauret. _"The Design of Web APIs"_
  
See an example at:  https://editor.swagger.io/

The following OpenAPI specification is the declarative equivalent of all the knowledge [`minitwit_sim_api.py`](https://github.com/itu-devops/2020-spring/blob/master/sessions/session_03/API%20Spec/minitwit_sim_api.py), [`minitwit_sim_api_test.py`](https://github.com/itu-devops/2020-spring/blob/master/sessions/session_03/API%20Spec/minitwit_sim_api_test.py), [`minitwit_simulator.py`](https://github.com/itu-devops/2020-spring/blob/master/sessions/session_03/API%20Spec/minitwit_simulator.py), and [`minitwit_scenario.csv`](https://github.com/itu-devops/2020-spring/blob/master/sessions/session_03/API%20Spec/minitwit_scenario.csv)


```yaml
openapi: 3.0.0
info:
  title: Minitwit Simualtor API
  description: This is a demo of Minitwit Simualtor API using OpenAPI spec
  contact:
    email: niss@itu.dk
  license:
    name: Apache 2.0
    url: http://www.apache.org/licenses/LICENSE-2.0.html
  version: 1.0.0
servers:
- url: /
paths:
  /latest:
    get:
      summary: Get latest accpeted id
      

      responses:
        "200":
          description: returns latest accepted id by api
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/LatestResponse'
  /register:
    post:
      summary: Post new user to register
      parameters:
      - name: latest
        in: query
        description: latest id sent by simulator api
        required: false
        style: form
        explode: true
        schema:
          type: integer
      requestBody:
        $ref: '#/components/requestBodies/RegisterUserBody'
      responses:
        "204":
          description: User registered
        "400":
          description: Error on insert with description
          content:
            application/json:
              schema:
                type: array
                items:
                  $ref: '#/components/schemas/RegisterErrorResponse'
  /msgs:
    get:
      summary: Get number of messages from system
      parameters:
      - name: Authorization
        in: header
        required: true
        style: simple
        explode: false
        schema:
          type: string
          example: Basic c2ltdWxhdG9yOnN1cGVyX3NhZmUh
      - name: "no"
        in: query
        description: Number of messages to return
        required: false
        style: form
        explode: true
        schema:
          type: integer
      - name: latest
        in: query
        description: latest id sent by simulator api
        required: false
        style: form
        explode: true
        schema:
          type: integer
      responses:
        "200":
          description: Message created
          content:
            application/json:
              schema:
                type: array
                items:
                  $ref: '#/components/schemas/ListOfMessageDTO'
  /msgs/{username}:
    get:
      summary: Find messages by username
      description: Returns messages by username
      parameters:
      - name: Authorization
        in: header
        required: true
        style: simple
        explode: false
        schema:
          type: string
          example: Basic c2ltdWxhdG9yOnN1cGVyX3NhZmUh
      - name: username
        in: path
        description: username of user to return
        required: true
        style: simple
        explode: false
        schema:
          type: string
          example: niss
      - name: "no"
        in: query
        description: pass an optional search string for looking up inventory
        required: false
        style: form
        explode: true
        schema:
          type: integer
      - name: latest
        in: query
        description: latest id sent by simulator api
        required: false
        style: form
        explode: true
        schema:
          type: integer
      responses:
        "200":
          description: successful operation
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/ListOfMessageDTO'
            application/xml:
              schema:
                $ref: '#/components/schemas/LatestResponse'
        "401":
          description: Unauthorized
        "404":
          description: User not found
    post:
      summary: Find messages by username
      description: Returns messages by username
      parameters:
      - name: Authorization
        in: header
        required: true
        style: simple
        explode: false
        schema:
          type: string
          example: Basic c2ltdWxhdG9yOnN1cGVyX3NhZmUh
      - name: username
        in: path
        description: username of user to return
        required: true
        style: simple
        explode: false
        schema:
          type: string
          example: niss
      - name: latest
        in: query
        description: latest id sent by simulator api
        required: false
        style: form
        explode: true
        schema:
          type: integer
      requestBody:
        $ref: '#/components/requestBodies/MessageBody'
      responses:
        "204":
          description: successful operation
        "401":
          description: Unauthorized
        "500":
          description: Invalid username supplied
  /fllws/{username}:
    get:
      summary: Find messages by username
      description: Returns messages by username
      parameters:
      - name: Authorization
        in: header
        required: true
        style: simple
        explode: false
        schema:
          type: string
          example: Basic c2ltdWxhdG9yOnN1cGVyX3NhZmUh
      - name: username
        in: path
        description: username of user to return
        required: true
        style: simple
        explode: false
        schema:
          type: string
          example: niss
      - name: "no"
        in: query
        description: pass an optional search string for looking up inventory
        required: false
        style: form
        explode: true
        schema:
          type: integer
      - name: latest
        in: query
        description: latest id sent by simulator api
        required: false
        style: form
        explode: true
        schema:
          type: integer
      responses:
        "200":
          description: successful operation
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/UserFollowersDTO'
        "401":
          description: Unauthorized
        "404":
          description: User not found
    post:
      summary: Find messages by username
      description: Returns messages by username
      parameters:
      - name: Authorization
        in: header
        required: true
        style: simple
        explode: false
        schema:
          type: string
          example: Basic c2ltdWxhdG9yOnN1cGVyX3NhZmUh
      - name: username
        in: path
        description: username of user to return
        required: true
        style: simple
        explode: false
        schema:
          type: string
          example: niss
      - name: latest
        in: query
        description: latest id sent by simulator api
        required: false
        style: form
        explode: true
        schema:
          type: integer
      requestBody:
        $ref: '#/components/requestBodies/FollowBody'
      responses:
        "204":
          description: successful operation
        "401":
          description: Unauthorized
        "404":
          description: User not found
components:
  schemas:
    LatestResponse:
      required:
      - latest
      type: object
      properties:
        latest:
          type: integer
          example: 23592
    RegisterErrorResponse:
      required:
      - error_msg
      - status
      type: object
      properties:
        status:
          type: integer
          example: 400
        error_msg:
          type: string
          example: error
    MessageDTO:
      required:
      - content
      - pub_date
      - user
      type: object
      properties:
        content:
          type: string
          example: Hello, up for coffee later?
        pub_date:
          type: string
          format: date-time
        user:
          type: string
          example: niss
    ListOfMessageDTO:
      required:
      - filtered_msgs
      type: object
      properties:
        filtered_msgs:
          type: array
          items:
            $ref: '#/components/schemas/MessageDTO'
    MessageCreateDTO:
      required:
      - content
      type: object
      properties:
        content:
          type: string
          example: Hey, whats up?
    FollowDTO:
      type: object
      properties:
        follow:
          type: string
          example: niss
    UnfollowDTO:
      type: object
      properties:
        unfollow:
          type: string
          example: niss
    UserFollowDTO:
      required:
      - username
      type: object
      properties:
        username:
          type: string
          example: niss
    UserFollowersDTO:
      required:
      - follows
      type: object
      properties:
        follows:
          type: array
          items:
            $ref: '#/components/schemas/UserFollowDTO'
    UserCreateDTO:
      required:
      - email
      - pwd
      - username
      type: object
      properties:
        username:
          type: string
          example: Christoffer Nissen
        email:
          type: string
          example: niss@itu.dk
        pwd:
          type: string
          example: securepwd
  requestBodies:
    FollowBody:
      content:
        application/json:
          schema:
            oneOf:
            - $ref: '#/components/schemas/FollowDTO'
            - $ref: '#/components/schemas/UnfollowDTO'
          example:
            follow: niss
    MessageBody:
      description: Message object that needs to be added to the database
      content:
        application/json:
          schema:
            $ref: '#/components/schemas/MessageCreateDTO'
      required: true
    RegisterUserBody:
      description: User object that needs to be added to the database
      content:
        application/json:
          schema:
            $ref: '#/components/schemas/UserCreateDTO'
      required: true
```




### How to design good web APIs?

  * M. Stocker et al. [_"Interface Quality Patterns: Communicating and Improving the Quality of Microservices APIs"_](https://dl.acm.org/doi/10.1145/3282308.3282319)
  * O. Zimmermann et al. [_"Interface Representation Patterns: Crafting and Consuming Message-Based Remote APIs"_](https://dl.acm.org/doi/10.1145/3147704.3147734)

### Your Turn! -  `Task 4`
<img src="https://media.giphy.com/media/13GIgrGdslD9oQ/giphy.gif" width=50%/>

Navigate to https://github.com/itu-devops/itu-minitwit-openapi and follow the scenario described in the `README.md` file.

-------


# Software Quality

## Pair programming

  > By  working  in tandem,  the  pairs  completed their  assignments 40% to 50% faster.
  >
  > [L. Williams et al. _"Strengthening the Case for Pair Programming"_](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=854064)
 
 

  > The top benefit was fewer bugs in the source code. One person 
said "it greatly reduces bug numbers." Simple bugs were found 
and fixed, as one respondent reported, "there are fewer 'petty'
bugs." In addition, respondents speculated that the longer bugs 
live in the code, the more difficult they are to fix. Using pair pro-
gramming, "bugs are spotted earlier" in the development process, 
and "may prevent bugs before [they are] deeply embedded."
  > 
  > A. Begel et al. _"Pair programming: what's in it for me?"_

  > for a development-time cost of about 15%, pair programming improves design quality, reduces defects, reduces staffing risk, enhances technical skills, improves team communications and is considered more enjoyable at statistically significant levels.
  >
  > https://collaboration.csc.ncsu.edu/laurie/Papers/XPSardinia.PDF



##  Code reviews
  
  > At a cost of 1-2% of the project, a 40% decrease in the number of issues  was  found.
  >
  > [R.A. Baker Jr  _"Code reviews enhance software quality"_](https://dl.acm.org/doi/pdf/10.1145/253228.253461)

  > Findings  show  that  unreviewed  commits  (i.e., commits  that did  not  undergo  a  review  process)  have  over  two  times  more chances of introducing bugs than reviewed commits (i.e.,commits that  underwent  a  review  process).  In  addition,  code  committed after  review  has  a  substantially  higher  readability  with  respect to  unreviewed  code.
  >
  > [G. Bavota et al. _"Four eyes are better than two: On the impact of code reviews on software quality"_](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.709.2980&rep=rep1&type=pdf)






  > we ﬁnd that both code review coverage and participation share a significant link with software quality. Low code review coverage and participation are estimated to produce components with up to two and five additional post-release defects respectively. Our results empirically confirm the intuition that poorly reviewed code has a negative impact on software quality [...]
  >
  > [S. McIntosh _"The impact of code review coverage and code review participation on software quality: a case study of the Qt, VTK, and ITK projects"_](https://dl.acm.org/doi/abs/10.1145/2597073.2597076)

## Static Analysis Tools


In the following we want to asses some qualities of an earlier version of _ITU-MiniTwit_, the one published in the master branch of https://github.com/itu-devops/itu-minitwit-monitoring.


### [Pylint](pylint.org)

  * checks coding standards 
  * detects errors
  * detects duplicates 
  * detects unused code
  

```
$ pylint minitwit_mysql.py
************* Module minitwit_mysql
minitwit_mysql.py:33:2: W0511: TODO change... (fixme)
minitwit_mysql.py:74:0: C0303: Trailing whitespace (trailing-whitespace)
minitwit_mysql.py:178:0: C0301: Line too long (114/100) (line-too-long)
minitwit_mysql.py:178:0: C0301: Line too long (114/100) (line-too-long)
minitwit_mysql.py:206:0: C0301: Line too long (101/100) (line-too-long)
minitwit_mysql.py:239:0: C0301: Line too long (107/100) (line-too-long)
minitwit_mysql.py:239:0: C0301: Line too long (107/100) (line-too-long)
minitwit_mysql.py:28:0: E0611: No name 'check_password_hash' in module 'werkzeug' (no-name-in-module)
minitwit_mysql.py:28:0: E0611: No name 'generate_password_hash' in module 'werkzeug' (no-name-in-module)
minitwit_mysql.py:48:0: C0103: Constant name "app" doesn't conform to UPPER_CASE naming style (invalid-name)
minitwit_mysql.py:79:4: C0103: Variable name "rv" doesn't conform to snake_case naming style (invalid-name)
minitwit_mysql.py:67:20: W0613: Unused argument 'args' (unused-argument)
minitwit_mysql.py:91:4: C0103: Variable name "rv" doesn't conform to snake_case naming style (invalid-name)
minitwit_mysql.py:309:0: E1101: Method 'jinja_env' has no 'filters' member (no-member)
minitwit_mysql.py:310:0: E1101: Method 'jinja_env' has no 'filters' member (no-member)
minitwit_mysql.py:13:0: W0611: Unused import sqlite3 (unused-import)
minitwit_mysql.py:16:0: W0611: Unused closing imported from contextlib (unused-import)

-----------------------------------
Your code has been rated at 7.54/10
```

  * R ... Refactor to adhere to "good practice"
  * C ... Convention violation of coding standard
  * W ... Warning indicating style issues or minor programming issues 
  * E ... Error, most likely bugs
  * F ... Fatal, errors which prevent further analysis

### [Pyflakes](https://github.com/PyCQA/pyflakes)

  * detects errors

```
$ pyflakes minitwit_mysql.py
minitwit_mysql.py:13: 'sqlite3' imported but unused
minitwit_mysql.py:16: 'contextlib.closing' imported but unused
```

### [Prospector ](https://github.com/PyCQA/prospector)

Prospector is a "meta-tool" as it includes since it incorporates results from other static analysis tools, such as, [Pylint](http://docs.pylint.org/), [pep8](http://pep8.readthedocs.org/en/latest/), and [McCabe complexity](https://pypi.python.org/pypi/mccabe), etc.


```
$ prospector minitwit_mysql.py
Messages
========

minitwit_mysql.py
  Line: 13
    pylint: unused-import / Unused import sqlite3
  Line: 16
    pylint: unused-import / Unused closing imported from contextlib
  Line: 39
    dodgy: password / Possible hardcoded password
  Line: 45
    dodgy: secret / Possible hardcoded secret key
  Line: 67
    pylint: unused-argument / Unused argument 'args' (col 20)



Check Information
=================
         Started: 2020-03-11 17:41:13.781662
        Finished: 2020-03-11 17:41:16.566612
      Time Taken: 2.78 seconds
       Formatter: grouped
        Profiles: default, no_doc_warnings, no_test_warnings, strictness_medium, strictness_high, strictness_veryhigh, no_member_warnings
      Strictness: None
  Libraries Used: flask
       Tools Run: dodgy, mccabe, pep8, profile-validator, pyflakes, pylint
  Messages Found: 5
```


### [Bandit](https://github.com/PyCQA/bandit)

Tries to detect

  * security defects
  * hardcoded passwords
  * Shell injections
  * SQL injections

```
$ bandit -r minitwit_mysql.py
[main]	INFO	profile include tests: None
[main]	INFO	profile exclude tests: None
[main]	INFO	cli include tests: None
[main]	INFO	cli exclude tests: None
[main]	INFO	running on Python 3.7.3
[node_visitor]	INFO	Unable to find qualified name for module: minitwit_mysql.py
Run started:2020-03-11 16:44:27.375384

Test results:
>> Issue: [B105:hardcoded_password_string] Possible hardcoded password: 'root'
   Severity: Low   Confidence: Medium
   Location: minitwit_mysql.py:39
   More Info: https://bandit.readthedocs.io/en/latest/plugins/b105_hardcoded_password_string.html
38	DB_USER = "root"
39	DB_PASSWORD = "root"
40	DB_NAME = "minitwit"

--------------------------------------------------
>> Issue: [B105:hardcoded_password_string] Possible hardcoded password: 'development key'
   Severity: Low   Confidence: Medium
   Location: minitwit_mysql.py:45
   More Info: https://bandit.readthedocs.io/en/latest/plugins/b105_hardcoded_password_string.html
44	DEBUG = True
45	SECRET_KEY = "development key"
46
47	# create our little application :)
48	app = Flask(__name__)

--------------------------------------------------
>> Issue: [B608:hardcoded_sql_expressions] Possible SQL injection vector through string-based query construction.
   Severity: Medium   Confidence: Medium
   Location: minitwit_mysql.py:89
   More Info: https://bandit.readthedocs.io/en/latest/plugins/b608_hardcoded_sql_expressions.html
88	    cur.execute(
89	        "select user_id from user where username = '{}'".format(username)
90	    )

--------------------------------------------------
>> Issue: [B303:blacklist] Use of insecure MD2, MD4, MD5, or SHA1 hash function.
   Severity: Medium   Confidence: High
   Location: minitwit_mysql.py:104
   More Info: https://bandit.readthedocs.io/en/latest/blacklists/blacklist_calls.html#b303-md5
103	    return "http://www.gravatar.com/avatar/%s?d=identicon&s=%d" % (
104	        md5(email.strip().lower().encode("utf-8")).hexdigest(),
105	        size,

--------------------------------------------------
>> Issue: [B608:hardcoded_sql_expressions] Possible SQL injection vector through string-based query construction.
   Severity: Medium   Confidence: Low
   Location: minitwit_mysql.py:118
   More Info: https://bandit.readthedocs.io/en/latest/plugins/b608_hardcoded_sql_expressions.html
117	        g.user = query_db(
118	            "select * from user where user_id = '{}'".format(session["user_id"]),
119	            one=True,

--------------------------------------------------
>> Issue: [B608:hardcoded_sql_expressions] Possible SQL injection vector through string-based query construction.
   Severity: Medium   Confidence: Low
   Location: minitwit_mysql.py:147
   More Info: https://bandit.readthedocs.io/en/latest/plugins/b608_hardcoded_sql_expressions.html
146	                                    where who_id = {}))
147	        order by message.pub_date desc limit {}"""
148	            .format(session["user_id"], session["user_id"], PER_PAGE),

--------------------------------------------------
>> Issue: [B608:hardcoded_sql_expressions] Possible SQL injection vector through string-based query construction.
   Severity: Medium   Confidence: Low
   Location: minitwit_mysql.py:161
   More Info: https://bandit.readthedocs.io/en/latest/plugins/b608_hardcoded_sql_expressions.html
160	        where message.flagged = 0 and message.author_id = user.user_id
161	        order by message.pub_date desc limit {}""".format(PER_PAGE)),
162	    )

--------------------------------------------------
>> Issue: [B608:hardcoded_sql_expressions] Possible SQL injection vector through string-based query construction.
   Severity: Medium   Confidence: Low
   Location: minitwit_mysql.py:169
   More Info: https://bandit.readthedocs.io/en/latest/plugins/b608_hardcoded_sql_expressions.html
168	    profile_user = query_db(
169	        "select * from user where username = '{}'".format(username), one=True
170	    )

--------------------------------------------------
>> Issue: [B608:hardcoded_sql_expressions] Possible SQL injection vector through string-based query construction.
   Severity: Medium   Confidence: Low
   Location: minitwit_mysql.py:178
   More Info: https://bandit.readthedocs.io/en/latest/plugins/b608_hardcoded_sql_expressions.html
177	                """select 1 from follower where
178	            follower.who_id = {} and follower.whom_id = {}""".format(session["user_id"], profile_user["user_id"]),
179	                one=True,

--------------------------------------------------
>> Issue: [B608:hardcoded_sql_expressions] Possible SQL injection vector through string-based query construction.
   Severity: Medium   Confidence: Low
   Location: minitwit_mysql.py:189
   More Info: https://bandit.readthedocs.io/en/latest/plugins/b608_hardcoded_sql_expressions.html
188	            user.user_id = message.author_id and user.user_id = {}
189	            order by message.pub_date desc limit {}""".format(profile_user["user_id"], PER_PAGE),
190	        ),

--------------------------------------------------
>> Issue: [B608:hardcoded_sql_expressions] Possible SQL injection vector through string-based query construction.
   Severity: Medium   Confidence: Medium
   Location: minitwit_mysql.py:206
   More Info: https://bandit.readthedocs.io/en/latest/plugins/b608_hardcoded_sql_expressions.html
205	    cur.execute(
206	        "insert into follower (who_id, whom_id) values ({}, {})".format(session["user_id"], whom_id),
207	    )

--------------------------------------------------
>> Issue: [B608:hardcoded_sql_expressions] Possible SQL injection vector through string-based query construction.
   Severity: Medium   Confidence: Medium
   Location: minitwit_mysql.py:223
   More Info: https://bandit.readthedocs.io/en/latest/plugins/b608_hardcoded_sql_expressions.html
222	    cur.execute(
223	        "delete from follower where who_id={} and whom_id={}".format(session["user_id"], whom_id),
224	    )

--------------------------------------------------
>> Issue: [B608:hardcoded_sql_expressions] Possible SQL injection vector through string-based query construction.
   Severity: Medium   Confidence: Medium
   Location: minitwit_mysql.py:239
   More Info: https://bandit.readthedocs.io/en/latest/plugins/b608_hardcoded_sql_expressions.html
238	            """insert into message (author_id, text, pub_date, flagged)
239	            values ({}, '{}', {}, 0)""".format(session["user_id"], request.form["text"], int(time.time())),
240	        )

--------------------------------------------------
>> Issue: [B608:hardcoded_sql_expressions] Possible SQL injection vector through string-based query construction.
   Severity: Medium   Confidence: Low
   Location: minitwit_mysql.py:255
   More Info: https://bandit.readthedocs.io/en/latest/plugins/b608_hardcoded_sql_expressions.html
254	            """select * from user where
255	            username = '{}'""".format(request.form["username"]),
256	            one=True,

--------------------------------------------------
>> Issue: [B608:hardcoded_sql_expressions] Possible SQL injection vector through string-based query construction.
   Severity: Medium   Confidence: Low
   Location: minitwit_mysql.py:289
   More Info: https://bandit.readthedocs.io/en/latest/plugins/b608_hardcoded_sql_expressions.html
288	            query = """insert into user (
289	                username, email, pw_hash) values ('{}', '{}', '{}')""".format(
290	                    request.form["username"],

--------------------------------------------------
>> Issue: [B104:hardcoded_bind_all_interfaces] Possible binding to all interfaces.
   Severity: Medium   Confidence: Medium
   Location: minitwit_mysql.py:314
   More Info: https://bandit.readthedocs.io/en/latest/plugins/b104_hardcoded_bind_all_interfaces.html
313	if __name__ == "__main__":
314	    app.run(host="0.0.0.0", port=5000)

--------------------------------------------------

Code scanned:
	Total lines of code: 251
	Total lines skipped (#nosec): 0

Run metrics:
	Total issues (by severity):
		Undefined: 0.0
		Low: 2.0
		Medium: 14.0
		High: 0.0
	Total issues (by confidence):
		Undefined: 0.0
		Low: 8.0
		Medium: 7.0
		High: 1.0
Files skipped (0):
```

### [Black](https://github.com/psf/black)

  > Black is the uncompromising Python code formatter. By using it, you agree to cede control over minutiae of hand-formatting. In return, Black gives you speed, determinism, and freedom from `pycodestyle` nagging about formatting. You will save time and mental energy for more important matters.
  >
  > https://github.com/psf/black
  
  
```
$ black minitwit_mysql.py
reformatted minitwit_mysql.py
All done! ✨ 🍰 ✨
1 file reformatted.
```

The above tool presentation is inspired by https://luminousmen.com/post/python-static-analysis-tools

-------

# Student Group Presentations


![](https://thumbs.gfycat.com/UniqueGeneralFieldmouse.webp)


  * `<master>` (group a)
  * `2k20 MSc` (group b)
  * `U buntu?` (group c)
  * `Fuld smadder` (group h)
  * `LazyOps - Semi manual CI/CD` (group e)
  * `noname` (group j)
  * `Blanao` (group l)
  

-----------


# Your turn now!

<img src="https://media.giphy.com/media/13GIgrGdslD9oQ/giphy.gif" width=50%/>

  - [1) Refactoring for Clean Subsystem Interfaces](#1\)-Refactoring-for-Clean-Subsystem-Interfaces)
  - [2) Enhance your CI Pipelines with at least two static analysis tools](#2\)-Enhance-your-CI-Pipelines-with-at-least-two-static-analysis-tools)
  - [3) Software Maintenance](#2\)-Software-Maintenance)


## 1) Refactoring for Clean Subsystem Interfaces

Your task until next week is to refactor at least one central interface in between sub-systems of your _ITU-MiniTwit_. Take either of the two examples from class (interface within programming language or web API) as foundation of your refactoring.

## 2) Enhance your CI Pipelines with at least two static analysis tools

Choose and include at least two static analysis tools into your CI/CD pipelines. You might want to start searching for relevant tools for your respective languages and frameworks here: https://github.com/mre/awesome-static-analysis.

Include them in a way that you either abort build in case quality of your code is too low (you will have to define some threshold here) or that you decide to trust them and let the tools try to automatically fix problems (in case you find a tool that supports that).

In the latter case, you want top let the corresponding tool contribute the applied changes back to the corresponding source repository.

## 3) Software Maintenance 

We are in maintenance. That is, fix issues of your version of _ITU-MiniTwit_ as soon as possible. Either issues that you find via your monitoring or those that you observe under: http://142.93.104.18/status.html